### Отбор признаков

#### Одномерные статистики

In [16]:
from sklearn.datasets import load_breast_cancer
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

dataset = load_breast_cancer()
#print(dataset.DESCR)

In [18]:
#Задаём начальные значения рандома
rng = np.random.RandomState(42)
#Добавляем шумовые признаки, сгенерированные просто так
noise = rng.normal(size=(len(dataset.data), 50))
X_w_noise = np.hstack([dataset.data, noise])

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_w_noise, dataset.target, random_state=1234, test_size=.5)
#Отберем 50% признаков 
select = SelectPercentile(percentile=50)
select.fit(X_train, y_train)
X_train_selected = select.transform(X_train)

In [29]:
#Выясним, что это за признаки
mask = select.get_support()
print(mask)

[ True  True  True  True  True  True  True  True  True False  True False
  True  True  True  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True False  True False False  True  True
 False False False False False  True False False False False False  True
 False  True False False False False False False False False  True False
  True False False False False False False  True False False False False
  True  True  True False False False  True False]


In [34]:
#Изучим правильность работы со всеми и выбранными признаками в отдельности
from sklearn.ensemble import RandomForestClassifier
X_test_selected = select.transform(X_test)
rf = RandomForestClassifier(random_state=1234)
rf.fit(X_train, y_train)
print("Правильность со всеми признаками: {}".format(rf.score(X_test, y_test)))
rf.fit(X_train_selected, y_train)
print("Правильность с выбранными признаками: {}".format(rf.score(X_test_selected, y_test)))

Правильность со всеми признаками: 0.9192982456140351
Правильность с выбранными признаками: 0.9333333333333333


#### Признаки на основе модели

In [38]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
select = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=1234), threshold='median')
select.fit(X_train, y_train)
X_train_l1 = select.transform(X_train)
#Выясним, что это за признаки
mask = select.get_support()
print(mask)

[ True  True  True  True  True  True  True  True  True  True  True False
  True  True False  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True False False False False  True  True
 False  True  True False False False  True False False False  True False
 False False False False  True False False False  True  True False  True
  True False  True False False False False False False False False False
 False False False False False False False  True]


In [40]:
X_test_l1 = select.transform(X_test)
score = RandomForestClassifier().fit(X_train_l1, y_train).score(X_test_l1, y_test)
print("Правильность с выбранными признаками: {}".format(score))

Правильность с выбранными признаками: 0.9263157894736842


#### Итеративный отбор признаков

In [42]:
from sklearn.feature_selection import RFE
select = RFE(RandomForestClassifier(n_estimators=100, random_state=1234), n_features_to_select=40)
select.fit(X_train, y_train)
#Выясним, что это за признаки
mask = select.get_support()
print(mask)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True False  True False  True False  True  True  True  True
  True  True  True  True  True  True False  True False False  True  True
 False  True  True False False False False False  True False  True False
 False False False False  True False False False False False  True  True
 False False  True False False False False False False False False  True
 False  True False False False False False False]


In [43]:
X_train_rfe = select.transform(X_train)
X_test_rfe = select.transform(X_test)
score = RandomForestClassifier().fit(X_train_rfe, y_train).score(X_test_rfe, y_test)
print("Правильность с выбранными признаками: {}".format(score))

Правильность с выбранными признаками: 0.9263157894736842
